# Causal Inference

## data preparation

In [1]:
# Start writing code here...

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
path = ""
# from google.colab import drive 
# drive.mount('/content/drive',force_remount=True)
# path = "/content/drive/MyDrive/209 Project/"

Mounted at /content/drive


In [10]:
##---------------Mobility Data-----------------------
sd_data = pd.read_csv(path + "data/distance_data/state_2020.csv")
#sd_data['fip'] = sd_data['county_fips'].apply(lambda x: str(int(x)).zfill(5))

print(sd_data.shape)
sd_data.head(3)

# Calculate ratio of different types of trips
sd_data["trip_ratio"] = sd_data["trips"]/sd_data["pop"]

sd_agg = sd_data[['date',  'trip_ratio',
                    "weekday","avg_stay_at_home_ratio","state_code"]]
#print(sd_agg.shape)
sd_agg.head(2)


(15555, 24)


,date,trip_ratio,weekday,avg_stay_at_home_ratio,state_code
0,2020-01-01,2.973107,2,0.253960,NH
1,2020-01-01,2.846696,2,0.233074,CA


In [12]:
##---------------Covid Data-----------------------
daily_cases = pd.read_csv(path + 'data/covid_cases/covid_confirmed_usafacts_new.csv')
daily_cases['fip'] = daily_cases['countyFIPS'].apply(lambda x: str(x).zfill(5))
daily_cases = daily_cases[1:]


daily_cases.head(2)

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,fip
1,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1805,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911,1924,1928,1949,1966,1983,1989,1999,2010,2021,2023,2030,2048,2059,2074,2082,2103,2126,2141,2159,2173,2186,2197,2212,2230,2242,2267,2283,2304,01001
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6024,6048,6073,6085,6116,6134,6141,6172,6190,6203,6220,6248,6270,6285,6333,6350,6369,6375,6405,6443,6475,6615,6637,6658,6694,6712,6743,6768,6888,6940,6966,6985,6995,7061,7097,7134,7188,7226,7263,01003


In [13]:
# convert original data to format ['fip', 'County Name', 'State', 'date', 'daily_confirmed']

# get daily confirmed per row
tmp = daily_cases.copy()
cols = daily_cases.columns[4:-1]# daily cases data
tmp['daily_confirmed'] = tmp[cols].values.tolist()
# tmp['time_list'] = [list(cols)]*tmp.shape[0]

# convert list of pd.Series then stack it
tmp = (tmp
 .set_index(['fip','County Name','State'])['daily_confirmed']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .rename(columns={0:'daily_confirmed'}))


tmp2 = daily_cases.copy()
cols = daily_cases.columns[4:-1]
tmp2['time_list'] = [list(cols)]*tmp2.shape[0]

tmp2 = (tmp2
 .set_index(['fip','County Name','State'])['time_list']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .rename(columns={0:'old_date'}))

cases_daily = pd.merge(tmp,tmp2,on=list(tmp.columns[:4]))
del tmp,tmp2,daily_cases

# convert date format to be consistent with social distance data
def date_in_sd(case_date):
    tmp = '2020-'+case_date.split('/')[0].zfill(2)+'-'+ case_date.split('/')[1].zfill(2)
    return tmp

cases_daily['date'] = cases_daily['old_date'].apply(date_in_sd)
cases_daily = cases_daily[['fip', 'County Name', 'State', 'date', 'daily_confirmed']]
print(cases_daily.shape)
cases_daily.head(2)

(935842, 5)


,fip,County Name,State,date,daily_confirmed
0,01001,Autauga County,AL,2020-01-22,0
1,01001,Autauga County,AL,2020-01-23,0


In [14]:
cases_daily["new_cases"] = cases_daily.groupby(by = "fip")["daily_confirmed"].diff()
cases_agg = cases_daily.copy()

for i in range(1,8):
    col_name = "confirmed_lag_%s"%(i)
    cases_agg[col_name] = cases_agg.groupby('fip')['new_cases'].shift(i)

print(cases_agg.shape)
cases_agg[cases_agg["date"] == "2020-05-22"].head(2)

(935842, 13)


,fip,County Name,State,date,daily_confirmed,new_cases,confirmed_lag_1,confirmed_lag_2,confirmed_lag_3,confirmed_lag_4,confirmed_lag_5,confirmed_lag_6,confirmed_lag_7
121,01001,Autauga County,AL,2020-05-22,149,2.0,11.0,9.0,7.0,10.0,0.0,7.0,0.0
414,01003,Baldwin County,AL,2020-05-22,271,1.0,0.0,8.0,2.0,6.0,0.0,10.0,1.0


In [15]:
##---------------Population Data-----------------------
population = pd.read_csv(path + "data/covid_cases/covid_county_population_usafacts.csv")
population = population[(population['countyFIPS'] != 0) & (population['countyFIPS'] != 6000)]
population['population_scale'] = population['population']/10000
population_state = population.groupby(['State'])['population_scale'].sum().to_frame(name = 'population').reset_index()
population_state.head()

,State,population
0,AK,73.1545
1,AL,490.3185
2,AR,301.7804
3,AZ,727.8717
4,CA,3951.2223


In [18]:
##---------------Policy Data-----------------------
#read and preprocess policy data
policy_df = pd.read_csv(path + "data/policy_data/policy_final_new.csv")
policy_df = policy_df.fillna(0).drop(policy_df.columns[0], axis = 1)

#drop non-informational features
policy_df = policy_df.drop(policy_df.columns[policy_df.columns.to_series().str.contains('-')], axis = 1)
policy_df = policy_df.drop('Emergency Declaration_Yes', axis = 1)
policy_df = policy_df.drop(policy_df.columns[policy_df.columns.to_series().str.contains('Large Gatherings Ban')], axis = 1)
policy_df = pd.merge(policy_df, population_state, on = "State")
policy_df.head()


,Date,State,Location,Stay at Home Order_Lifted,Stay at Home Order_Rolled Back to High Risk Groups,Stay at Home Order_Statewide,Mandatory Quarantine for Travelers_All Air Travelers,Mandatory Quarantine for Travelers_All Travelers,Mandatory Quarantine for Travelers_From Certain States,Mandatory Quarantine for Travelers_Lifted,Mandatory Quarantine for Travelers_Rolled Back to Certain States,Mandatory Quarantine for Travelers_Rolled Back to International Travel,Restaurant Limits_Closed Except for Takeout/Delivery,Mandatory Quarantine for Travelers_From Certain States (New),Restaurant Limits_New Capacity Limits,Restaurant Limits_New Service Limits,Stay at Home Order_New Stay at Home Order,Mandatory Quarantine for Travelers_All Travelers (New),population
0,2020-06-04,AK,Alaska,1,0,0,0.0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,73.1545
1,2020-06-05,AK,Alaska,1,0,0,0.0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,73.1545
2,2020-06-06,AK,Alaska,1,0,0,0.0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,73.1545
3,2020-06-07,AK,Alaska,1,0,0,0.0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,73.1545
4,2020-06-08,AK,Alaska,1,0,0,0.0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,73.1545


In [19]:
##---------------final Data-----------------------
#create final data
cases_sd_state = pd.DataFrame()
#drop correlated trip_ratio data
sd_agg1 = sd_agg
cases_sd_state = cases_agg.groupby(['State', 'date'])['daily_confirmed', 'new_cases'].sum().reset_index()
cases_sd_state.head()

#merge behavior, policy, cases data
d1 = pd.merge(cases_sd_state, sd_agg1, left_on = ["State", "date"], right_on = ["state_code","date"])
data = pd.merge(d1,policy_df, left_on = ["State", "date"], right_on = ["State","Date"])
#data = pd.merge(data, test_df[["state","date", 'totalTest_log_delta']], left_on = ["State", "date"], right_on = ["state","date"])

#create month and weekday dummies
data['month'] = pd.DatetimeIndex(data['date']).month
data = pd.concat([data,pd.get_dummies(data['month'], prefix='month')],axis=1).drop('month', axis=1)
data = pd.concat([data,pd.get_dummies(data['weekday'], prefix='weekday')],axis=1).drop('weekday', axis=1)
data = data.drop(['state_code', 'Date', 'Location'], axis = 1)
data = data[data['new_cases'] >= 0]
data['new_cases_scale'] = np.log(data['new_cases']+ 1)

#data = data.dropna()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


## Indirect Influence: Behavior VS Policy

In [20]:
# Model: trip ratio VS policies
features = list(data.columns[7:]) #+ ['new_cases_scale']
X_train = data[(data['date']<'20201001')][features]
y_train = data[(data['date']<'20201001')]['trip_ratio']
X_train = sm.add_constant(X_train)
lr_b_p_trip = sm.OLS( y_train, X_train).fit()
trip_df = lr_b_p_trip.params.reset_index().iloc[:-11]
print(lr_b_p_trip.summary())

# Model: stay at home ratio VS policies
X_train = data[(data['date']<'20201001')][features]
y_train = data[(data['date']<'20201001')]['avg_stay_at_home_ratio']
X_train = sm.add_constant(X_train)
lr_b_p_sah = sm.OLS( y_train, X_train).fit()
sah_df = lr_b_p_sah.params.reset_index().iloc[:-11]
print(lr_b_p_sah.summary())

trip_pvalue = lr_b_p_trip.pvalues.reset_index()
sah_pvalue = lr_b_p_sah.pvalues.reset_index()
trip_pvalue.columns = ["feature", "trip_pvalue"]
sah_pvalue.columns = ["feature", "sah_pvalue"]


                            OLS Regression Results                            
Dep. Variable:             trip_ratio   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     242.3
Date:                Sun, 13 Dec 2020   Prob (F-statistic):               0.00
Time:                        16:54:29   Log-Likelihood:                -1078.9
No. Observations:                7639   AIC:                             2210.
Df Residuals:                    7613   BIC:                             2390.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

## Direct Influence: Growth rate VS Policy & Behavior

In [21]:
# Calculate growth rate of 14 days later
data['growth_rate_14'] = data.groupby('State')['new_cases'].shift(-14)/data.groupby('State')['daily_confirmed'].shift(-13)
data1 = data.dropna()


In [22]:
#
features = list(data.columns[4:-1]) #+['new_cases_scale']
X_train1 = data1[(data1['date']<'2020-10-01')][features]
y_train1 = data1[(data1['date']<'2020-10-01')]['growth_rate_14']
print(X_train1.shape)
print(y_train1.shape)
X_train1 = sm.add_constant(X_train1)
lr_y_b_p = sm.OLS( y_train1, X_train1).fit()
lr_y_b_p.summary()


(6059, 31)
(6059,)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         growth_rate_14   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     119.2
Date:                Sun, 13 Dec 2020   Prob (F-statistic):               0.00
Time:                        16:54:34   Log-Likelihood:                 19589.
No. Observations:                6059   AIC:                        -3.912e+04
Df Residuals:                    6031   BIC:                        -3.893e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================================
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------
const                                                                      0.0286      0.001     19.246      0.000       0.026       0.031
trip_ratio                                                                -0.0101      0.000    -23.143      0.000      -0.011      -0.009
avg_stay_at_home_ratio                                                    -0.0035      0.005     -0.726      0.468      -0.013       0.006
Stay at Home Order_Lifted                                                 -0.0021      0.000     -4.638      0.000      -0.003      -0.001
Stay at Home Order_Rolled Back to High Risk Groups                        -0.0022      0.001     -3.093      0.002      -0.004      -0.001
Stay at Home Order_Statewide                                              -0.0031      0.001     -3.871      0.000      -0.005      -0.002
Mandatory Quarantine for Travelers_All Air Travelers                       0.0057      0.001      5.124      0.000       0.004       0.008
Mandatory Quarantine for Travelers_All Travelers                           0.0021      0.000      4.496      0.000       0.001       0.003
Mandatory Quarantine for Travelers_From Certain States                     0.0070      0.001     10.279      0.000       0.006       0.008
Mandatory Quarantine for Travelers_Lifted                                  0.0044      0.000     14.001      0.000       0.004       0.005
Mandatory Quarantine for Travelers_Rolled Back to Certain States          -0.0006      0.001     -0.805      0.421      -0.002       0.001
Mandatory Quarantine for Travelers_Rolled Back to International Travel    -0.0033      0.001     -3.477      0.001      -0.005      -0.001
Restaurant Limits_Closed Except for Takeout/Delivery                      -0.0068      0.001     -6.907      0.000      -0.009      -0.005
Mandatory Quarantine for Travelers_From Certain States (New)              -0.0039      0.001     -6.294      0.000      -0.005      -0.003
Restaurant Limits_New Capacity Limits                                     -0.0094      0.001     -7.079      0.000      -0.012      -0.007
Restaurant Limits_New Service Limits                                      -0.0043      0.000     -8.895      0.000      -0.005      -0.003
Stay at Home Order_New Stay at Home Order                                 -0.0049      0.002     -2.594      0.009      -0.009      -0.001
Mandatory Quarantine for Travelers_All Travelers (New)                  3.114e-19   6.75e-19      0.462      0.644   -1.01e-18    1.63e-18
population                                                             -2.642e-06   2.47e-07    -10.709      0.000   -3.13e-06   -2.16e-06
month_6                  

## Get Total Effect of policy

In [23]:
#calculate the total effect of policy
param_df = lr_y_b_p.params.reset_index().iloc[:-14]
trip_df.columns = ["feature", "trip_coef"]
sah_df.columns = ["feature", "sah_coef"]
param_df.columns = ["feature", "coef"]
p_df = lr_y_b_p.pvalues.reset_index()
p_df.columns = ['feature', 'pvalue']


dfs = [trip_df, sah_df, trip_pvalue, sah_pvalue, p_df]
policy_coef = param_df.copy()

for i in dfs:
    policy_coef = pd.merge(policy_coef,i, how = "left", on = "feature" )


policy_coef['total_effect'] = policy_coef['coef'] + policy_coef['trip_coef']*policy_coef.iloc[1,1]
+ policy_coef['sah_coef']*policy_coef.iloc[2,1]
policy_coef = policy_coef.iloc[4:]
policy_coef = policy_coef.sort_values(by=['total_effect', 'pvalue', 'trip_pvalue', 'sah_pvalue'])
policy_coef

,feature,coef,trip_coef,sah_coef,trip_pvalue,sah_pvalue,pvalue,total_effect
12,Restaurant Limits_Closed Except for Takeout/De...,-6.843949e-03,2.143751e-01,1.085687e-02,7.581401e-14,5.831631e-05,5.439737e-12,-9.009015e-03
14,Restaurant Limits_New Capacity Limits,-9.389594e-03,-2.502676e-01,3.644898e-02,6.899029e-11,9.095505e-24,1.617549e-12,-6.862033e-03
4,Stay at Home Order_Rolled Back to High Risk Gr...,-2.209300e-03,3.470964e-01,1.125424e-02,1.561884e-122,1.994415e-16,1.988093e-03,-5.714776e-03
13,Mandatory Quarantine for Travelers_From Certai...,-3.894042e-03,1.100972e-01,5.234472e-02,1.555598e-14,1.235543e-300,3.312044e-10,-5.005960e-03
15,Restaurant Limits_New Service Limits,-4.331246e-03,-4.958424e-02,1.385844e-02,3.044826e-05,8.982017e-35,7.624271e-19,-3.830474e-03
11,Mandatory Quarantine for Travelers_Rolled Back...,-3.251569e-03,-7.515711e-03,-2.664023e-03,7.386623e-01,2.100150e-01,5.098583e-04,-3.175665e-03
10,Mandatory Quarantine for Travelers_Rolled Back...,-5.847164e-04,1.477342e-01,3.721797e-04,1.123705e-16,8.243820e-01,4.209767e-01,-2.076747e-03
16,Stay at Home Order_New Stay at Home Order,-4.882554e-03,-2.880755e-01,6.604246e-04,3.816548e-12,8.657836e-01,9.498385e-03,-1.973156e-03
5,Stay at Home Order_Statewide,-3.090788e-03,-1.920956e-01,2.873342e-02,1.074003e-26,8.374203e-64,1.093618e-04,-1.150733e-03
17,Mandatory Quarantine for Travelers_All Travele...,3.113975e-19,-1.009320e-16,-5.233647e-18,1.244423e-22,6.927772e-08,6.443621e-01,1.330753e-18


In [24]:
# get important/unimportant policies based on their effects and pvalue
df_important = policy_coef[((policy_coef['pvalue'] < 0.05) | (policy_coef['trip_pvalue'] < 0.05) | 
              (policy_coef['sah_pvalue'] < 0.05)) 
                            & (policy_coef['total_effect'] < 0)].sort_values(by = "total_effect")[["feature", "pvalue", "trip_pvalue"
                                                                               ,"sah_pvalue", 'total_effect']]

all_policies = set(policy_coef['feature'])
important_policies = set(df_important['feature'])
unimportant_policies = all_policies-important_policies
print(unimportant_policies)

print(important_policies)
print(df_important)

{'Mandatory Quarantine for Travelers_All Travelers', 'Mandatory Quarantine for Travelers_Lifted', 'Mandatory Quarantine for Travelers_From Certain States', 'Mandatory Quarantine for Travelers_All Travelers (New)', 'Mandatory Quarantine for Travelers_All Air Travelers'}
{'Restaurant Limits_New Capacity Limits', 'Stay at Home Order_Statewide', 'Stay at Home Order_New Stay at Home Order', 'Mandatory Quarantine for Travelers_Rolled Back to International Travel', 'Restaurant Limits_New Service Limits', 'Restaurant Limits_Closed Except for Takeout/Delivery', 'Mandatory Quarantine for Travelers_Rolled Back to Certain States', 'Stay at Home Order_Rolled Back to High Risk Groups', 'Mandatory Quarantine for Travelers_From Certain States (New)'}
                                              feature  ...  total_effect
12  Restaurant Limits_Closed Except for Takeout/De...  ...     -0.009009
14              Restaurant Limits_New Capacity Limits  ...     -0.006862
4   Stay at Home Order_Rolled Back t

In [25]:
#get counterfactual effect of polocy on growth rate
policy_df['Restaurant'] = -9.009015e-03*np.where(policy_df.groupby(by = "State")['Restaurant Limits_Closed Except for Takeout/Delivery'].shift(14)==1, -1, 1)
policy_df['SAH'] = -5.005960e-03*np.where(policy_df.groupby(by = "State")['Stay at Home Order_Rolled Back to High Risk Groups'].shift(14)==1, -1, 1)
policy_df['Quarantine'] = -5.714776e-03*np.where(policy_df.groupby(by = "State")['Mandatory Quarantine for Travelers_From Certain States (New)'].shift(14)==1, -1, 1)
policy_causal_df = policy_df[['Date', 'State', 'Restaurant', 'SAH', 'Quarantine']]
policy_causal_df.columns = ['Date', 'State', 'Restaurant Limits_Closed Except for Takeout/Delivery', 'Stay at Home Order_Rolled Back to High Risk Groups', 'Mandatory Quarantine for Travelers_From Certain States (New)']
policy_causal_df.to_csv('policy_causal_df.csv')
policy_causal_df



,Date,State,Restaurant Limits_Closed Except for Takeout/Delivery,Stay at Home Order_Rolled Back to High Risk Groups,Mandatory Quarantine for Travelers_From Certain States (New)
0,2020-06-04,AK,-0.009009,-0.005006,-0.005715
1,2020-06-05,AK,-0.009009,-0.005006,-0.005715
2,2020-06-06,AK,-0.009009,-0.005006,-0.005715
3,2020-06-07,AK,-0.009009,-0.005006,-0.005715
4,2020-06-08,AK,-0.009009,-0.005006,-0.005715
...,...,...,...,...,...
8305,2020-11-09,AL,-0.009009,-0.005006,-0.005715
8306,2020-11-10,AL,-0.009009,-0.005006,-0.005715
8307,2020-11-11,AL,-0.009009,-0.005006,-0.005715
8308,2020-11-12,AL,-0.009009,-0.005006,-0.005715


In [26]:
policy_coef.to_csv(path + "data/policy_coef.csv")